<a href="https://colab.research.google.com/github/amyfthau/CFGSum23D1G4/blob/data_collection/ONS_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Google Drive and Change Directory

Getting started with the API
Our datasets endpoint lists all the datasets we have available on the beta API.

https://api.beta.ons.gov.uk/v1/datasets

https://digitalblog.ons.gov.uk/2021/02/15/how-to-access-data-from-the-ons-beta-api/

How to query the API
There are three ways to query the ONS API:

1. Download the entire dataset.
2. Query an observation.
3. Filter a dataset for more advanced queries.

To download the entire CPIH dataset, navigate to the latest_version. From here you will see the CSV or XLS download link. You can read it directly into a program such as Python or paste the URL into a web browser to download it automatically.

**The second way to query the API is the observations endpoint. This allows you to query an observation or several observations of data. To write our query we will need to know what dimensions are contained within the CPIH dataset and select at least one option from each.**

**To do this, take the latest_version URL and add /dimensions**

In [ ]:
# Import relevant libraries to mount drive
from google.colab import drive
import os

# Mount google drive
drive.mount('/content/drive')

# Define the path to your specific folder
specific_folder_path = '/content/drive/MyDrive/CFGdegree_Sum23_D1_G4'

# Navigate to the specific folder
os.chdir(specific_folder_path)

# change directory to shared group folder
%cd /content/drive/MyDrive/CFGdegree_Sum23_D1_G4




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content


Import relevant Libraries

In [ ]:
import pandas as pd
import requests

Checking ONS API

note: The Office for National Statistics API makes datasets and other data available programmatically using HTTP. It allows you to filter datasets and directly access specific data points.

In [ ]:
# The URL we want to request data from
ONS_base_url =  'https://api.beta.ons.gov.uk/v1/datasets'
# Request data from the URL
response = requests.get(ONS_base_url)
# Print the request
print(response)
# check if the request was successful (HTTP status code 200)
if response.status_code == 200:
  # convert response to JSON (dictionary)
    ONS_data_response = response.json()
    print(ONS_data_response)
else:
    print('Request failed with status code:', response.status_code)


<Response [200]>
{'@context': 'https://cdn.ons.gov.uk/assets/json-ld/context.json', 'count': 20, 'items': [{'contacts': [{'email': 'qualityoflife@ons.gov.uk', 'name': 'Rachel Mullis, Emily Froud and Geeta Kerai', 'telephone': '+44 300 0671543'}], 'description': 'Seasonally and non seasonally-adjusted quarterly estimates of life satisfaction, feeling that the things done in life are worthwhile, happiness and anxiety in the UK.', 'id': 'wellbeing-quarterly', 'keywords': ['well-being'], 'links': {'editions': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/wellbeing-quarterly/editions'}, 'latest_version': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/wellbeing-quarterly/editions/time-series/versions/7', 'id': '7'}, 'self': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/wellbeing-quarterly'}, 'taxonomy': {'href': 'https://api.beta.ons.gov.uk/v1/peoplepopulationandcommunity/wellbeing'}}, 'methodologies': [{'href': 'https://www.ons.gov.uk/peoplepopulationandcommunity/wellbeing/method

Obviously we need to try to get something more specific

In [ ]:
# Lets loop over the data retreived to see if we can break it down
for key in ONS_data_response:
  print(key)
  print("")



@context

count

items

limit

offset

total_count



This information isnt useful - apart from that we need to go inside the items part

In [ ]:
# access the 'items' list
items_list = ONS_data_response['items']
#print(items_list)

# Loop over dictionaries to see what it looks like
for item in items_list:
  print(item.keys(),"\n" ,item.items())


Okay, that looks useful but the format is making it difficult to extract from

In [ ]:
# Loop over keys to see what it looks like
for item in items_list:
  print(item.keys(),"\n")


In [ ]:
print(len(items_list))


20


In [ ]:
# Loop over dictionaries to see what it looks like
count = 0
for item in items_list:
  print("***********")
  print(f"Count: {count}")
  print(f"ID: {item['id']} \nTitle: {item['title']} \nDescription: {item['description']}")
  print("***********")
  count +=1


Okay, now we have a better idea of how this works - lets try their example endpoint...

/datasets/cpih01/editions/time-series/versions/6/observations?time=Oct-11&geography=K02000001&aggregate=cpih1dim1A0


In [ ]:
# Construct the API request URL
# ONS_endpoint = '/datasets/cpih01/editions/time-series/versions/6/observations?time=Oct-11&geography=K02000001&aggregate=cpih1dim1A0'
# ONS_URL = ONS_base_url + ONS_endpoint



# Make the API request
response = requests.get(ONS_URL)

if response.status_code == 200:
    # Convert response to JSON (dictionary)
    ONS_data_response = response.json()
    print(ONS_data_response)

    # # Get the CSV download link from the response
    # csv_download_link = ONS_data_response['downloads']['csv']['href']

    # # Make a new request to download the CSV data
    # csv_response = requests.get(csv_download_link)

    # if csv_response.status_code == 200:
    #     # Save the CSV data to a file
    #     with open('cpih_data.csv', 'wb') as csv_file:
    #         csv_file.write(csv_response.content)
    #         print('CSV data saved successfully.')
    # else:
    #     print('Failed to download CSV data with status code:', csv_response.status_code)
else:
    print('Request failed with status code:', response.status_code)

{'@context': 'https://cdn.ons.gov.uk/assets/json-ld/context.json', 'alerts': [], 'collection_id': 'cmdcpih-2dd77bce9c8d5d08703147b3d2b27ab78ab927c4f38c9c1f2b4369f3bb165f4a', 'dimensions': [{'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mmm-yy', 'id': 'mmm-yy', 'label': 'Time', 'links': {'code_list': {}, 'options': {}, 'version': {}}, 'name': 'time'}, {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/uk-only', 'id': 'uk-only', 'label': 'Geography', 'links': {'code_list': {}, 'options': {}, 'version': {}}, 'name': 'geography'}, {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/cpih1dim1aggid', 'id': 'cpih1dim1aggid', 'label': 'Aggregate', 'links': {'code_list': {}, 'options': {}, 'version': {}}, 'name': 'aggregate'}], 'downloads': {'csv': {'href': 'https://download.beta.ons.gov.uk/downloads/datasets/cpih01/editions/time-series/versions/6.csv', 'size': '5148315'}, 'csvw': {'href': 'https://download.beta.ons.gov.uk/downloads/datasets/cpih01/editions/time-series/versions/6.csv-meta

In [ ]:
# Define the base URL
ONS_base_url = "https://api.beta.ons.gov.uk/v1"
dataset_id = "cpih01"
edition = "time-series"
version = "6"

# To do this, take the latest_version URL and add /dimensions.
# To find out what options are available for each dimension, choose one and add it to your request, followed by /options.
dimension_name = "time"
dimension_value = "Oct-11"
# Define query parameters/labels
geography = "K02000001"
aggregate = "cpih1dim1A0"

# Construct the API request URL
ONS_endpoint = f"/datasets/{dataset_id}/editions/{edition}/versions/{version}/observations"
query_params = {
    "geography": geography,
    "aggregate": aggregate,
    dimension_name: dimension_value
}

#/datasets/{id}/editions/{edition}/versions/{version}/observations
# /datasets/{datasetId}/editions/{edition}/versions/{version}/observations?time={timeLabel}&geography={geographyID}&dimension3={dimension3ID}&dimension4={dimension4ID}...

response = requests.get(ONS_base_url + ONS_endpoint, params=query_params)

# Process the response
if response.status_code == 200:
    ONS_data_response = response.json()
    observation = ONS_data_response["observations"][0]["observation"]
    print("Observation:", observation)
else:
    print('Request failed with status code:', response.status_code)


Observation: {'observation': '94.5'}


In [ ]:
# Define the base URL
ONS_base_url = "https://api.beta.ons.gov.uk/v1"
dataset_id = "cpih01"
edition = "time-series"
version = "6"

# To do this, take the latest_version URL and add /dimensions.
# To find out what options are available for each dimension, choose one and add it to your request, followed by /options.
dimension_name = "time"
dimension_value = "*"
# Define query parameters/labels
geography = "K02000001"
aggregate = "cpih1dim1A0"

# Construct the API request URL
ONS_endpoint = f"/datasets/{dataset_id}/editions/{edition}/versions/{version}/observations"
query_params = {
    "geography": geography,
    "aggregate": aggregate,
    dimension_name: dimension_value
}

#/datasets/{id}/editions/{edition}/versions/{version}/observations
# /datasets/{datasetId}/editions/{edition}/versions/{version}/observations?time={timeLabel}&geography={geographyID}&dimension3={dimension3ID}&dimension4={dimension4ID}...

response = requests.get(ONS_base_url + ONS_endpoint, params=query_params)

# Create lists to store data
dates = []
data_values = []

# Process the response
if response.status_code == 200:
    ONS_data_response = response.json()
    observations = ONS_data_response["observations"]

    # this part helps us look at the structure
    # print("Observation:", observation)

    for observation in observations:
      # # Print each observation and date
      # # print out whole data, mm-yy: xx.x (float)
      # print(f"{observation['dimensions']['Time']['label']}: {observation['observation']}")

      # retreives data and appends it into preallocated variables
      date = observation['dimensions']['Time']['label']
      data = float(observation['observation'])
      dates.append(date)
      data_values.append(data)

    # Create a dataframe
    df = pd.DataFrame({'Date': dates, 'Data': data_values})
    print(df)

else:
    print('Request failed with status code:', response.status_code)


       Date   Data
0    May-04   77.9
1    May-13   98.2
2    Sep-16  101.5
3    May-88   48.0
4    Apr-18  105.5
..      ...    ...
393  Nov-90   56.9
394  Aug-12   96.1
395  Aug-20  108.8
396  Mar-19  107.0
397  May-20  108.6

[398 rows x 2 columns]
